In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Spotify API credentials
client_id = 'your_client_id'
client_secret = 'your_client_secret'
redirect_uri = 'http://localhost:8888/callback'

# Initialize Spotify API with OAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope="user-library-read user-top-read"))

def get_user_top_tracks_artists(username):
    # Get user's top tracks and artists
    top_tracks = sp.current_user_top_tracks(limit=10)
    top_artists = sp.current_user_top_artists(limit=10)
    
    return top_tracks, top_artists

def recommend_songs(username):
    # Get the user's top tracks and artists
    top_tracks, top_artists = get_user_top_tracks_artists(username)

    # Extract track names and artist names
    top_track_names = [track['name'] for track in top_tracks['items']]
    top_artist_names = [artist['name'] for artist in top_artists['items']]

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(top_track_names + top_artist_names)

    # Compute the cosine similarity between TF-IDF vectors
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Get recommendations by finding less popular songs similar to the user's top tracks and artists
    recommendations = []
    for i, track in enumerate(top_track_names):
        similar_indices = cosine_similarities[i].argsort()[:-1:-1]  # Sort in descending order
        for idx in similar_indices:
            if track != top_track_names[idx]:  # Exclude the same track
                recommendations.append((top_track_names[idx], top_artist_names[idx]))

    return recommendations

if __name__ == "__main__":
    # Replace 'your_username' with the user's Spotify username
    username = 'your_username'
    
    recommendations = recommend_songs(username)
    
    print("Recommended Songs:")
    for i, (track, artist) in enumerate(recommendations):
        print(f"{i + 1}. {track} by {artist}")
